# Project Proposal: Group 59
### Jin Kim, Emily Ishii, Natasha Larin, Syed Haque

### Introduction

For our DSCI 100 Project, our group will be using the data found in the "processed.cleveland.data" file in the Heart Disease dataset directory. This dataset is composed of 14 fields:

* Age of patient (age) - years
* Sex of the patient (sex)
    * 1 = Male
    * 2 = Female
* Chest pain type (cp)
    * 1 = Typical Angina
    * 2 = Atypical Angina
    * 3 = Non-aginal Pain
    * 4 = Asymptomatic
* Blood preassure (trestbps) - mm Hg
* Serum cholestoral (chol) - mg/dl
* Fasting blood sugar > 120 mg/dl (fbs) - true or false
* Resting ECG results (restecg)
    * 0 = Normal
    * 1 = Abnormal
    * 2 = Probable or definite left ventricular hypertrophy
* Maximum heart rate achieved (thalach) - bpm
* Excercize induced agina (exang)
    * 0 = No
    * 1 = Yes
* Depression induced by excercize related to rest (oldpeak)
* Slope of the peake excercise segment (slope)
* Number of major vessels (ca)
* Condition (thal)
    * 3 = Normal
    * 6 = fixed defect
    * 7 = reversable defect
* Diagnosis of heart disease (num)
    * idk what the numbers mean
    
These fields are parts of a patient's health report, so they can be used to predict the severity or existence of heart disease within a patient, which is what variable **num** represents.

The question that we will be exploring is: **Which factors are most influential towards the cause of heart disease?**

### Preliminary Exploratory Data Analysis

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

**Reading in, tidying, and splitting the dataset**

In [11]:
names <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")
patient_data <- read_csv("data/processed.cleveland.data", col_names = names) |>
    mutate(num = as_factor(num)) |>
    mutate(cp = as_factor(cp)) |>
    mutate(sex = as_factor(sex)) |>
    mutate(exang = as_factor(exang)) |>
    mutate(slope = as_factor(slope)) |>
    mutate(restecg = as_factor(restecg)) |>
    mutate(ca = as_factor(ca)) |>
    mutate(thal = as_factor(thal)) |>
    mutate(fbs = as.logical(fbs)) |>
    select(age, trestbps, chol, thalach, oldpeak, num) # Since we are classifying using these variables, we should select only continuous variables (data of type double)
patient_split <- initial_split(patient_data, prop = 0.75, strata = num)
training_data <- training(patient_split)
testing_data <- testing(patient_split)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


**Summarizing the training dataset**

In [12]:
data_mean <- training_data |>
    select(-num) |>
    map_df(mean) |>
    pivot_longer(cols = age:oldpeak,
               names_to = "predictor",
               values_to = "mean") |>
    select(-predictor)
data_max <- training_data |>
    select(-num) |>
    map_df(max) |>
    pivot_longer(cols = age:oldpeak,
               names_to = "predictor",
               values_to = "max")
data_min <- training_data |>
    select(-num) |>
    map_df(min) |>
    pivot_longer(cols = age:oldpeak,
               names_to = "predictor",
               values_to = "min") |>
    select(-predictor)
training_data_summary <- bind_cols(data_max, data_min, data_mean)
training_data_summary

predictor,max,min,mean
<chr>,<dbl>,<dbl>,<dbl>
age,77.0,29,54.475556
trestbps,200.0,100,131.684444
chol,564.0,126,249.075556
thalach,202.0,71,149.848889
oldpeak,6.2,0,1.011556


**Visualizing the training dataset**

In [13]:
# Code for histogram visualization of each variable (probably should use facet_grid())

### Methods

How will we conduct our data analysis?
* Include prediciton/classification
* Analyze the behaviour of each variable
* Create a prediction model based on the information above
    * Evaluate the accuracy: If it is fairly accurate then those variables are the answer to our main question!
* Use visualizations! Maybe just 1 or 2?
    * I think we should visualize the data for the actual training data, and then on the same plot visualize what our prediction model predicts to compare the differences (hopefully its not that different)

### Expected outcomes and significance

What do we expect to find?
* All of the predictor variables that we used are important/influencial to determining if a patient has heart disease
Impacts of our findings:
* If our model is fairly good, then we can predict if a patient has a disease so that they may seek help immediately.
* However, our model will undoubtedly be less than 100% accurate, so instead of it being a complete scientific diagnosis, it can act as an indicator for concern (spark suspicion concerning the patient's health).
Future questions?
* What other variables could increase the accuracy of this model?
* Can this model be used to predict other health problems?
* How can we incorporate categoric variables into a prediction model? 